In [ ]:
%pip install sacrebleu

In [ ]:
import os
import pandas as pd
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPEN_ROUTER_KEY")

In [ ]:
bouquet_test = pd.read_parquet("bouquet_test_ce_overall.parquet")

In [ ]:
bouquet_test

,ce,ru,en,ru2ce_anthropic/claude-haiku-4.5,ce2ru_anthropic/claude-haiku-4.5,en2ce_anthropic/claude-haiku-4.5,ce2en_anthropic/claude-haiku-4.5,ru2ce_google/gemini-3-flash-preview,ce2ru_google/gemini-3-flash-preview,en2ce_google/gemini-3-flash-preview,ce2en_google/gemini-3-flash-preview,ce2ru_nllb,ru2ce_nllb,ce2en_nllb,en2ce_nllb,ce2ru_madlad,ru2ce_madlad,ce2en_madlad,en2ce_madlad
0,Хӏокхунах хӏун доьху?,Сколько это стоит?,How much is this?,Цуьнан сайна баьлла?,Как дела?,Mында itla nakhcha du?,How are you?,ХӀокхуьнан хьакъ (мах) мел бу?,Что из этого следует?,ХIара хIун мах болуш ду?,What is the price of this?,А чего он желает?,И йо?,"""Что просишь ты от этого?""",хара ду?,Что делать с этим?,Цуьнан мах мел бу?,What's this?,ХӀара мел ду?
1,"Мах мел бу хаа, соьга сайга йазде","Пишите в личку, чтобы узнать цену.",DM me for price,"Шуна язъе, цена белгаш йиш йолу.",Я не знаю точное значение этого текста. Предст...,Нохчийн меттан хилар дахьалар хаам.,"Let me go to that place, to write something th...","Мах хаа лаахь, личке язъе.","Напиши мне, сколько это стоит.","Мах хаа лиъча, соьга язде.",Write to me personally to let me know the price.,"сколько стоит знать, напиши мне","Мах хаьа, кехат т яздийша.",-Wanted? -Wanted? -Wanted? -Wanted? -Wanted?,махна соьга дахьедар,"Я знаю, сколько стоит, напишу себе",Меха хаа личка тӀе язде.,"Knowing the price, I write myself",мах доьхуш соьга дӀаязвала
2,Кхузахь схьаала!,Скажите здесь!,just say it here!,Бохьа ахийла ду!,Добро пожаловать!,Эвла ду ахь лаа!,Hello!,Кхузахь алахь!,Скажи это здесь!,кхузахь схьаала иза!,Speak it here!,подойди сюда!,Дийцийша кхузахь!,Watch!,кхузахь ала иза!,скажи здесь!,Кхузахь ала!,Come here!,кхузахь ала!
3,"Хьомениг, Хьан духар цӏе йу-кх!","Дорогая, твой наряд - огонь.","The outfit is fire, love.","Машар, сен сая хилла дукъ юкъ.","Хозяин, хан вам честь и уважение!","Costumer вай дика, дахар.","Hello, may God grant you good health!","ЙоI, хьан духар цIе ю.","Дорогая, у тебя наряд просто огонь!","Духар чӀогӀа хаза ду хьан, езар.","My dear, your outfit is red!",Возлюбленный! одеяние Твое чисто!,"Сан хьоме доттагӀ, хьан духар ю.","О, возлюбленная! - Твое одеяние - светильник.","И духар ю, безам.",Ибо одежда твоя - огонь!,"Сан хьомениг, хьан духар - цӀе ю.","""Friend, your clothing is fire!""","Юхар цӀе ю, безам бу."
4,"Баркалла, хьоьга а иштта.","Спасибо, взаимно.","Thank you, likewise.","Сахьа, я баран дика.","Спасибо, приходи еще.","Сахьа, сахьа.","Thank you, and may God bless you.","Баркалла, хьуна а (шуна а) хуьлда иштта.","Спасибо, и тебе (вам) тоже.","Баркалла, хьуна а хуьлда иштта.","Thank you, the same to you.","Спасибо, и тебе тоже.",Баркалла вовшашна.,"Спасибо, а также и вам.",баркалла ас хьоьга а.,Спасибо тебе тоже.,"Баркалла, вовшашна а.","Thank you, too.",Шуна а баркалла.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,"Цундела, билггал оцу контекстехь кхета веза де...",Поэтому свободу слова следует понимать именно ...,"Therefore, freedom of expression must be under...","Себ мотт, сацам свободе шуна йогIаш декъашлла ...","Контекст, в котором развивается общество, треб...","Şuonçа, сацдина гулам ду хаамаш йуьхьалла ойла...",# Translation:\n\nContext is important in this...,Цундела дош аьлла дIахезийтаран маршо нисса оц...,Поэтому свободу слова следует понимать именно ...,"Цундела, дешан маршо оцу контекстехь кхето еза...","Therefore, freedom of speech must be understoo...","В силу этого, в конкретном контексте необходим...","Оццу контексте дийца дешан маршонан маьна, аьл...","It is, therefore, в данном контексте, целесооб...","Цундела, хокху контексте къаьсташ даран маршон...",Поэтому именно в этом контексте следует понима...,Цундела къамелан паргӀатло хӀокху контекстехь ...,"Therefore, the freedom of speech must be under...","Цундела, хӀокху контекстехь кхета деза вистхил..."
850,"Нагахь санна и саннарг хуьлуш меттиг нислахь, ...","Если такое происходит, то виновный в первую оч...","If these occur, the person who utters them is 

In [ ]:
import sacrebleu

bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)

In [ ]:
models = [
    "anthropic/claude-haiku-4.5",
    "google/gemini-3-flash-preview",
    "nllb",
    "madlad",
]
directions = [
    ("ce", "ru"),
    ("ru", "ce"),
    ("ce", "en"),
    ("en", "ce"),
]

report = {}

for model in models:
    print(f"Evaluation of {model}")
    report[model] = {}
    for src_lang, tgt_lang in directions:
        report[model][f"{src_lang}2{tgt_lang}"] = {}
        translations = bouquet_test[f"{src_lang}2{tgt_lang}_{model}"].tolist()
        references = [bouquet_test[tgt_lang].tolist()]
        report[model][f"{src_lang}2{tgt_lang}"]["BLEU"] = bleu_calc.corpus_score(
            translations, references
        )
        report[model][f"{src_lang}2{tgt_lang}"]["chrF2++"] = chrf_calc.corpus_score(
            translations, references
        )

print(report)

Evaluation of anthropic/claude-haiku-4.5
Evaluation of google/gemini-3-flash-preview
Evaluation of nllb
Evaluation of madlad
{'anthropic/claude-haiku-4.5': {'ce2ru': {'BLEU': BLEU = 3.12 20.4/3.8/1.6/0.8 (BP = 1.000 ratio = 1.373 hyp_len = 15928 ref_len = 11597), 'chrF2++': chrF2++ = 21.80}, 'ru2ce': {'BLEU': BLEU = 1.98 18.9/2.4/0.9/0.4 (BP = 1.000 ratio = 1.064 hyp_len = 14151 ref_len = 13302), 'chrF2++': chrF2++ = 17.22}, 'ce2en': {'BLEU': BLEU = 2.42 19.3/3.3/1.2/0.5 (BP = 1.000 ratio = 1.481 hyp_len = 19284 ref_len = 13023), 'chrF2++': chrF2++ = 22.20}, 'en2ce': {'BLEU': BLEU = 0.63 9.0/0.8/0.2/0.1 (BP = 1.000 ratio = 2.024 hyp_len = 26924 ref_len = 13302), 'chrF2++': chrF2++ = 11.75}}, 'google/gemini-3-flash-preview': {'ce2ru': {'BLEU': BLEU = 29.68 58.8/35.2/23.6/15.9 (BP = 1.000 ratio = 1.084 hyp_len = 12572 ref_len = 11597), 'chrF2++': chrF2++ = 56.40}, 'ru2ce': {'BLEU': BLEU = 9.69 42.0/14.2/6.0/2.7 (BP = 0.978 ratio = 0.978 hyp_len = 13013 ref_len = 13302), 'chrF2++': chrF2+

In [ ]:
report

{'anthropic/claude-haiku-4.5': {'ce2ru': {'BLEU': BLEU = 3.12 20.4/3.8/1.6/0.8 (BP = 1.000 ratio = 1.373 hyp_len = 15928 ref_len = 11597),
   'chrF2++': chrF2++ = 21.80},
  'ru2ce': {'BLEU': BLEU = 1.98 18.9/2.4/0.9/0.4 (BP = 1.000 ratio = 1.064 hyp_len = 14151 ref_len = 13302),
   'chrF2++': chrF2++ = 17.22},
  'ce2en': {'BLEU': BLEU = 2.42 19.3/3.3/1.2/0.5 (BP = 1.000 ratio = 1.481 hyp_len = 19284 ref_len = 13023),
   'chrF2++': chrF2++ = 22.20},
  'en2ce': {'BLEU': BLEU = 0.63 9.0/0.8/0.2/0.1 (BP = 1.000 ratio = 2.024 hyp_len = 26924 ref_len = 13302),
   'chrF2++': chrF2++ = 11.75}},
 'google/gemini-3-flash-preview': {'ce2ru': {'BLEU': BLEU = 29.68 58.8/35.2/23.6/15.9 (BP = 1.000 ratio = 1.084 hyp_len = 12572 ref_len = 11597),
   'chrF2++': chrF2++ = 56.40},
  'ru2ce': {'BLEU': BLEU = 9.69 42.0/14.2/6.0/2.7 (BP = 0.978 ratio = 0.978 hyp_len = 13013 ref_len = 13302),
   'chrF2++': chrF2++ = 35.83},
  'ce2en': {'BLEU': BLEU = 22.43 52.4/27.7/16.8/10.4 (BP = 1.000 ratio = 1.125 hyp_len